In [1]:
import numpy

In [4]:
# This function takes a square matrix and sets the new value of (row, col) to the old value of (col, col) - (row, col).
# So, when you feed it the matrix Utility(row | col), it gives you back the matrix Improvement(col,row | col)
def improve(m):
    blank = numpy.zeros(shape=(m.shape[0],m.shape[1]))
    for i in range(0,m.shape[0]):
        for j in range(0,m.shape[1]):
            blank[i][j] = m[j][j]-m[i][j]
    return blank

In [5]:
# This function takes a square matrix and sets the new value of (row, col) to the old value of (row, col) - (col, row).
# So, when you feed it the matrix Improvement(col, row | col), it gives you back the matrix News(col, row)
def newsify(m):
    blank = numpy.zeros(shape=m.shape)
    for row in range(0,m.shape[0]):
        for col in range(0,m.shape[1]):
            blank[row][col] = m[row][col] - m[col][row]
    return blank

In [6]:
# Here, I generate the matrix News(col, row) from the matrices payoff and prob provided above
News = newsify(improve(des.dot(probs)))

In [7]:
# Later, I need to know how many acts there are.  So I store the number of rows in News as "num_acts"
num_acts = News.shape[0]

In [8]:
# This function generates a list of ordered triples, where the first member of the triple is the column of News(col, row), 
# the second member is the row of News(col,row), and the third member is the value at (row, col).
# Because News(col, row) = - News(row, col), I only look at the upper triangle

def pairs(m):
    blank = []
    for row in range(0,m.shape[0]):
        for col in range(row+1,m.shape[1]):
            blank.append((col, row, m[row][col]))
    return blank

In [9]:
# Running the function "pairs" on the matrix "News"
pairs = pairs(News)

In [10]:
# This function allows me to convert the numbers from 0 to 25 into letters A--Z
def num_to_let(num):
    asciiInt = int(num) + 65
    letter = str(chr(asciiInt))
    return letter

In [11]:
# This replaces triples of the form (number, number, number) with ('letter', 'letter', number)
def letterify(l):
    blist = []
    for row in range(0,len(l)):
            blist.append((num_to_let(l[row][0]),num_to_let(l[row][1]),l[row][2]))
    return blist

In [12]:
# This list is the result of "letterifying" the list of pairs
init_list = letterify(pairs)

In [13]:
# This function sorts the tuples (option1, option2, strength) by the absolute value of "strength", in descending order
def Sort_Tuple(l):
    l.sort(key = lambda x:x[2],reverse=1)
    return l

In [14]:
# This list sorts the tuples by strength, in descending order
init_sorted_list = Sort_Tuple(init_list)

In [15]:
# This function checks to see whether there are ties in the "strength" of reasons 
def check_for_ties(l):
    answer = 0
    for i in range(0,len(l)):
        for j in range(i+1,len(l)):
            if abs(l[i][2])==abs(l[j][2]):
                answer = 1
                return answer
    return answer

In [16]:
def break_ties(l):
    if check_for_ties(l)==0:
        l.sort(key=lambda x:abs(x[2]),reverse=True)
    if check_for_ties(l)==1:
        print('Because there are ties in the strengths of your reasons, this decision requires the use of the tie-breaking algorithm.  You should enter some arbitrary preference between options which we will use to break ties.')
        print()
        first = input("What is your most preferred option?")
        if len(first)>1:
            first = input("Sorry, I didn't get that.  You should enter a single uppercase letter.")
        elif not (ord(first) >= 65 and ord(first) <= 90):
            first = input("Sorry, but you have to use uppercase letters.  Let's try that again.  Your most preferred options is:")
        my_dict = { first : 0 }
        count=1
        for num in range(1,num_acts):
            next = input("What is your next most preferred option?")
            if len(next)>1:
                next = input("Sorry, I didn't get that.  You should enter a single uppercase letter.")
            elif not (ord(next) >= 65 and ord(next) <= 90):
                next = input("Sorry, but you have to use uppercase letters.  Let's try that again.  Your next most preferred options is:")
            my_dict[next] = count
            count+=1
        l.sort(key=lambda x: my_dict[x[0]] if x[2]>=0 else my_dict[x[1]])
        l.sort(key=lambda x: my_dict[x[1]] if x[2]>=0 else my_dict[x[0]])
        l.sort(key=lambda x:abs(x[2]),reverse=True)
    return l

In [ ]:
break_ties(init_sorted_list)

In [18]:
# The 'purify' function swaps the position of the options if the strength is negative
def purify(l):
    blist = []
    for row in range(0,len(l)):
        if l[row][2] >0:
            blist.append((l[row][0], l[row][1],l[row][2]))
        elif l[row][2]<0:
            blist.append((l[row][1], l[row][0],abs(l[row][2])))
        elif l[row][2]==0:
            blist.append((l[row][0],l[row][1],l[row][2]))
    return blist

In [19]:
pure_pairs = purify(init_sorted_list)

In [20]:
#This function displays the preferences and strengths 
def pref_view_init(l):
    for entry in l:
        up = entry[0]
        down=entry[1]
        if entry[2]>0:
            print(up+'  >  '+down+'    ['+str(entry[2])+']')
        if entry[2]==0:
            print(up+'  =  '+down+'    ['+str(entry[2])+']')

In [21]:
#This function displays just the preferences
def pref_view(l):
    for entry in l:
        up = entry[0]
        down=entry[1]
        if entry[2]>0:
            print(up+'  >  '+down)
        if entry[2]==0:
            print(up+'  =  '+down)

In [ ]:
# Here are the tuples, ordered by the strength of the reason you have to prefer option1 to option2
print()
print("Here is the list of preferences which you have pro tanto reason to hold, ordered by the strength of the reason you have to hold them.")
print()
pref_view_init(pure_pairs)
print()
print("Here's how to read the list: 'X  >  Y    [s]' says that you have a pro tanto reason of strength s to prefer X to Y.  'X  =  Y   [0]' says that you have pro tanto reason to be indifferent between X and Y.")
print()

In [23]:
# With this function, we chain together any pairs where the 1st option matches the 2nd or the 2nd matches the first.
# Thus, X > Y and Y > Z leads to a pair X > Z
# And X > Y and Z = X leads to a pair Z > Y
# And X = Y and Y = Z leads to X = Z
def chain(l):
    blist = []
    for row2 in range(0,len(l)):
        for row1 in range(0,row2):
            if l[row1][2]!=0 or l[row2][2]!=0:
                if l[row1][0]==l[row2][1] and not (l[row2][0],l[row1][1],max(l[row1][2],l[row2][2])) in blist:
                    blist.append((l[row2][0],l[row1][1],max(l[row1][2],l[row2][2])))
                if l[row1][1]==l[row2][0] and not (l[row1][0],l[row2][1],max(l[row1][2],l[row2][2])) in blist:
                    blist.append((l[row1][0],l[row2][1],max(l[row1][2],l[row2][2])))
        blist.append(l[row2])
    return blist

In [24]:
# This function checks a list of preferences to see whether they form any cycles.
# It does this by running the "chain" function as many times as their are acts, effectively tracing out every possible path 
# through the options.  If any of these paths take you back to the option you started with, then the function 
# returns the answer "1".  Otherwise, it returns "0".
# Additionally, we check any indifferences in the list to see whether they conflict with a strict preference in the list
# For instance, X = Y conflicts with X > Y and Y > X
def check_for_cycles(l):
    blist = l
    answer = 0
    for num in range(0,num_acts+1):
        blist=chain(blist)
        for row in blist:
            if row[0]==row[1]:
                answer = 1
                return answer
            if row[2]==0:
                for row2 in blist:
                    if row2[2]>0:
                        if (row[0]==row2[0] and row[1]==row2[1]) or (row[0]==row2[1] and row[1]==row2[0]):
                            answer = 1
                            return answer                    
    return answer

In [25]:
# This function decides which strict preferences are retained by first adding the first two strict preferences to a list of 
# "keepers" (since the first two strict preferences cannot form a cycle on their own).  It then adds subsequent strict 
# preferences from the sorted list iff doing so does not lead to any cycles.  We ignore indifferences.
def kept_pairs(l):
    keep = []
    for row in range(0,len(l)):
        if l[row][2]!=0 and check_for_cycles(keep + [l[row]])==0:
            keep.append(l[row])
        if l[row][2]==0 and check_for_cycles(keep + [l[row]])==0 and check_for_cycles(keep + [[l[row][1],l[row][0],0]])==0:
            keep.append(l[row])
    return keep 

In [26]:
keepers = kept_pairs(pure_pairs)

In [ ]:
print()
print("After the algorithm is run, here are the preferences which are retained:")
print()
pref_view(keepers)
print()

In [28]:
# This function returns the options which are at the top of the preference ordering.  It scans through every pair of distinct
# strict preferences in a list and returns options which are strictly preferred to some option and not strictly *dis*preferred
# to any other option
def top(l):
    tops=[]
    for i in l:
        dispreferred=0
        if i[2]>0:
            for j in l:
                if i!=j and i[2] != 0 and j[2]!=0 and j[1]==i[0]:
                    dispreferred=1
            if dispreferred==0:
                tops.append(i[0])
        if i[2]==0:
            for j in l:
                if i!=j and j[2] != 0 and (j[1]==i[0] or j[1]==i[1]):
                    dispreferred=1
            if dispreferred==0:
                tops.append(i[0])
                tops.append(i[1])
    return tops

In [29]:
# This function removes any duplicates from a list.
def remove_duplicates(l):
    blist = [] 
    [blist.append(x) for x in l if x not in blist]
    return blist

In [30]:
def line_by_line(l):
    for item in l:
        print(item)

In [ ]:
print()
print('Therefore, the following options are permissible:')
print()
line_by_line(remove_duplicates(top(keepers)))